### To do:

- [x] get some extra **Python practice**
- [x] get lots of **tool practice**
- [x] increase your **Gradio proficiency**
- [ ] try **picture output** (on command only!)
- [ ] try **audio output** (on command only?)
- [ ] try **audio input?** (most importantly, do anything you saw in the video; the rest is nice-to-have)
_Extra: delve into Claude's function calling documentation (can be left for later)_

In [ ]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import random
import re

In [ ]:
load_dotenv()

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins with {openai_api_key[:8]}")
else:
    print("OpenAI API Key? As if!")
    
MODEL = "gpt-4o-mini"
openai = OpenAI()

In [ ]:
system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so. "

In [ ]:
# ticket price function

ticket_prices = {"valletta": "799 $", "turin": "899 $", "sacramento": "1400 $", "montreal": "499 $"} #awkward currency for better tts rendition

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")

In [ ]:
# travel booking function

def book_flight(destination_city):
    booking_code = ''.join(random.choice('0123456789BCDFXYZ') for i in range(2)) + ''.join(random.choice('012346789HIJKLMNOPQRS') for i in range(2)) + ''.join(random.choice('0123456789GHIJKLMNUOP') for i in range(2))
    print(f"Booking code {booking_code} generated for flight to {destination_city}.")
    
    return booking_code

In [ ]:
# verify if booking code is correct

def check_code(code):
    valid = "valid" if re.match("^[0123456789BCDFXYZ]{2}[012346789HIJKLMNOPQRS]{2}[0123456789GHIJKLMNUOP]{2}$", code) != None else "not valid"
    println(f"Code checker called for code {code}, which is {valid}.")
    return re.match("^[0123456789BCDFXYZ]{2}[012346789HIJKLMNOPQRS]{2}[0123456789GHIJKLMNUOP]{2}$", code) != None

In [ ]:
price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [ ]:
booking_function = {
    "name": "book_flight",
    "description": "Call this whenever you have to book a flight. Give it the destination city and you will get a booking code. Tell the customer \
that the flight is booked and give them the booking code obtained through this function. Never give any other codes to the customer.",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to book their flight to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [ ]:
codecheck_function = {
    "name": "check_code",
    "description": "Call this whenever you need to verify if a booking code for a flight (also called 'flight code', 'booking reference', \
or variations thereof) is valid.",
    "parameters": {
        "type": "object",
        "properties": {
            "code": {
                "type": "string",
                "description": "The code that you or the user needs to verify",
            },
        },
        "required": ["code"],
        "additionalProperties": False
    }
}

In [ ]:
# List of tools:

tools = [{"type": "function", "function": price_function}, {"type": "function", "function": booking_function}, {"type": "function", "function": codecheck_function}]

In [ ]:
def chat(history):
    messages = [{"role": "system", "content": system_message}] + history
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    image = None
    
    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        responses = handle_tool_call(message)
        messages.append(message)
        for response in responses:
            messages.append(response)
        response = openai.chat.completions.create(model=MODEL, messages=messages)
        
    reply = response.choices[0].message.content
    history += [{"role": "assistant", "content": reply}]
    
    return history, image

In [ ]:
def handle_tool_call(message):
    responses = []
    for tool_call in message.tool_calls:
        arguments = json.loads(tool_call.function.arguments)
        indata = arguments[list(arguments.keys())[0]] # works for now because we only have one argument in each of our functions
        function_name = tool_call.function.name
        if function_name == 'get_ticket_price':
            outdata = get_ticket_price(indata)
            input_name = "destination city"
            output_name = "price"
        elif function_name == 'book_travel':
            outdata = book_flight(indata)
            input_name = "destination city"
            output_name = "booking code"
        else:
            outdata = check_code(indata)
            input_name = "booking code"
            output_name = "validity"
        
        responses.append({
            "role": "tool",
            "content": json.dumps({input_name: indata, output_name: outdata}),
            "tool_call_id": tool_call.id
        })
    return responses

In [ ]:
# More involved Gradio code as we're not using the preset Chat interface!
# Passing in inbrowser=True in the last line will cause a Gradio window to pop up immediately.

with gr.Blocks() as ui:
    with gr.Row():
        chatbot = gr.Chatbot(height=500, type="messages")
        image_output = gr.Image(height=500)
    with gr.Row():
        entry = gr.Textbox(label="Chat with our AI Assistant:")
    with gr.Row():
        clear = gr.Button("Clear")

    def do_entry(message, history):
        history += [{"role":"user", "content":message}]
        return "", history

    entry.submit(do_entry, inputs=[entry, chatbot], outputs=[entry, chatbot]).then(
        chat, inputs=chatbot, outputs=[chatbot, image_output]
    )
    clear.click(lambda: None, inputs=None, outputs=chatbot, queue=False)

ui.launch(inbrowser=True)